**3D RECONSTRUCTION**

**Functions**

In [ ]:
# Calculando FPS
# Use -> start_time = time.time() # start time of the loop
def CalcFPS(start_time):
    fps = 'FPS: ' + str(int(1 / ((time.time() - start_time)/20)))
    
    return fps

# Load previously saved data of calibration
def LoadCalib():
    with np.load('data_calib_left.npz') as X:
        mtxL, distL, rvecsL, tvecsL, objpointsL, imgpointsL = [X[i] for i in ('mtxL', 'distL', 'rvecsL', 'tvecsL', 'objpointsL', 'imgpointsL')]
    with np.load('data_calib_right.npz') as X:
        mtxR, distR, rvecsR, tvecsR, objpointsR, imgpointsR = [X[i] for i in ('mtxR', 'distR', 'rvecsR', 'tvecsR', 'objpointsR', 'imgpointsR')]

    return mtxL, distL, rvecsL, tvecsL, objpointsL, imgpointsL, mtxR, distR, rvecsR, tvecsR, objpointsR, imgpointsR

# Apply Undistortion -- optional use
def ImageUndistort(img, mtx, dist):
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

    # Using undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    
    return dst

# Re-projection Errors -- optional use
def ProjError(imgpoints, objpoints, rvecs, tvecs, mtx, dist):
    tot_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
        tot_error += error
    
    errors = tot_error/len(objpoints)
    return errors

# Draw xyz axes -- other option of drawing
def DrawAxes(img, corners, rvecs, tvecs, mtx, dist):
    axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
    
    # project 3D points to image plane
    imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
    
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

# Draw Cube at axes
def DrawCube(img, corners, rvecs, tvecs, mtx, dist):
    axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])
    
    # project 3D points to image plane
    imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)  
    imgpts = np.int32(imgpts).reshape(-1,2)

    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)

    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)

    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)

    return img

# Set disparity parameters
def ConfigDisparity():
    
    window_size = 3
    min_disp = 16
    num_disp = 64 #divisible by 16
    
    stereoL = cv2.StereoSGBM_create(minDisparity = min_disp,
                                    numDisparities = num_disp,
                                    blockSize = window_size,
                                    uniquenessRatio = 10,
                                    speckleWindowSize = 100,
                                    speckleRange = 32,
                                    disp12MaxDiff = 5,
                                    P1 = 8*3*window_size**2,
                                    P2 = 32*3*window_size**2)
    
    # Used for the filtered image
    stereoR=cv2.ximgproc.createRightMatcher(stereoL) # Create another stereo for right this time
    
    return stereoL, stereoR
    
# Draw epipolar lines to adjsut camera
def DrawEpipolarLines(frameL, frameR):
    img_h, img_w, chl = frameL.shape
    
    for x in range(0, int(img_h/20)):
        cv2.line(frameL, (0,x*20), (img_w,x*20), (255,255,255), thickness=1, lineType=8, shift=0)
        cv2.line(frameR, (0,x*20), (img_w,x*20), (255,255,255), thickness=1, lineType=8, shift=0)

    return frameL, frameR
    
print('Functions OK')

**Main program**

In [ ]:
# DEPTH MAP FROM STEREO IMAGES
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

pt_col = 6
pt_lin = 5
count_time = 0
font = cv2.FONT_HERSHEY_SIMPLEX
fps = 'FPS: 30'
x_cut = 80
pic_count = 0

def nothing(x):
    pass
    
# Open cameras
CamR= cv2.VideoCapture(1)
CamL= cv2.VideoCapture(0)

# Load all calibration data
mtxL, distL, rvecsL, tvecsL, objpointsL, imgpointsL, mtxR, distR, rvecsR, tvecsR, objpointsR, imgpointsR = LoadCalib()
print('Data of calibration loaded with sucess!')

if CamR.isOpened() and CamL.isOpened():
    print('CAMERA ON')
else:
    print('CAMERA OFF')

while(True):
    if count_time == 0:
        start_time = time.time()
    
    # Read frames
    retL, frameL= CamL.read()
    retR, frameR= CamR.read()
    
    frameL_copy = frameL.copy()
    frameR_copy = frameR.copy()
    
    # Convert to grayscale
    grayL = cv2.cvtColor(frameL, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(frameR, cv2.COLOR_BGR2GRAY)
    
    #grayL = cv2.GaussianBlur(grayL, (5,5), 0)
    #grayR = cv2.GaussianBlur(grayR, (5,5), 0)
    
    # Undistort images
    imgL_und = ImageUndistort(grayL, mtxL, distL)
    imgR_und = ImageUndistort(grayR, mtxR, distR)
    
    # Resize images after undistorted
    imgL_resized = cv2.resize(imgL_und, (int(grayL.shape[1]), int(grayL.shape[0])))
    imgR_resized = cv2.resize(imgR_und, (int(grayR.shape[1]), int(grayR.shape[0])))    
  
    # Set disparity parameters and create matcher in both sides
    stereo_left, stereo_right = ConfigDisparity()
    
    # Calcule disparity
    disparityL = stereo_left.compute(imgL_resized, imgR_resized)
    disparityR = stereo_right.compute(imgR_resized, imgL_resized)

    # Filter parameters
    lmbda = 80000
    sigma = 1.2
    visual_multiplier = 1.0
    wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=stereo_left)
    wls_filter.setLambda(lmbda)
    wls_filter.setSigmaColor(sigma)
    
    # Filter disparity
    img_filtered = wls_filter.filter(disparityL, imgL_resized, None, disparityR)
    img_filtered = cv2.normalize(src=img_filtered, dst=img_filtered, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX);
    img_filtered = np.uint8(img_filtered)
    disparityL = np.uint8(disparityL)
    disparityR = np.uint8(disparityR)
    
    # Apply colormap to result
    img_colored = cv2.applyColorMap(img_filtered, cv2.COLORMAP_JET)
       
    # Draw epipolar lines to align cameras
    frameL, frameR = DrawEpipolarLines(frameL, frameR)
    
    # Calcule FPS rate and draw
    count_time += 1
    if count_time == 20:
        fps = CalcFPS(start_time)
        count_time = 0
    cv2.putText(frameL, fps, (10,15), font, 0.5, (255,255,255), 1, cv2.LINE_AA)

    cv2.imshow('disparity', img_colored)      
    cv2.imshow('frameL',frameL)
    cv2.imshow('frameR',frameR)
    
    # Take a picture with spacebar
    if cv2.waitKey(10) & 0xFF == 32:
        pic_count += 1
        file_name_L = 'outputs/frameL' + str(pic_count) + '.jpg'
        file_name_R = 'outputs/frameR' + str(pic_count) + '.jpg'
        file_name_res = 'outputs/disparity' + str(pic_count) + '.jpg'
        cv2.imwrite(file_name_L, frameL_copy[0:img_filtered.shape[0], x_cut:img_filtered.shape[1]])
        cv2.imwrite(file_name_R, frameR_copy[0:img_filtered.shape[0], x_cut:img_filtered.shape[1]])
        cv2.imwrite(file_name_res, img_filtered[0:img_filtered.shape[0], x_cut:img_filtered.shape[1]])
        
        plt.imshow(img_filtered[0:img_filtered.shape[0], x_cut:img_filtered.shape[1]])
        plt.show()
    
    # End the Programme
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
            
# When everything done, release the capture
CamR.release()
CamL.release()
cv2.destroyAllWindows()

**EPIPOLAR GEOMETRY**

In [ ]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c,_ = img1.shape
    #img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    #img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

def DrawAvgPoints(img1, img2, pts1, pts2):

    img_center1 = img1.copy()
    img_center2 = img2.copy()
    
    x_center_pts1 = (sum(pts1[:,0]) / len(pts1)).astype(int)
    y_center_pts1 = (sum(pts1[:,1]) / len(pts1)).astype(int)
    x_center_pts2 = (sum(pts2[:,0]) / len(pts2)).astype(int)
    y_center_pts2 = (sum(pts2[:,1]) / len(pts2)).astype(int)
    
    center_pts1 = (x_center_pts1, y_center_pts1)
    center_pts2 = (x_center_pts2, y_center_pts2)
    
    img_center1 = cv2.circle(img_center1, center_pts1, 5, (0,255,0), -1)
    img_center2 = cv2.circle(img_center2, center_pts2, 5, (0,255,0), -1)
    
    return img_center1, img_center2, center_pts1, center_pts2

print('Functions OK')

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob

center_pts1_arr = []
center_pts2_arr = []
pts1_arr = []
pts2_arr = []

images_left = []
images_left = glob.glob('outputs/frameL*.jpg')

for frame in range(0, len(images_left), 1):
    
    img1 = cv2.imread(images_left[0])  #queryimage # left image
    img2 = cv2.imread(images_left[frame]) #trainimage # right image
    print(images_left[0], ' ', images_left[frame])

    img1_copy = img1.copy()
    img2_copy = img2.copy()

    #sift = cv2.SIFT()
    sift = cv2.xfeatures2d.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)

    good = []
    pts1 = []
    pts2 = []

    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)

    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)

    # We select only inlier points
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]

    # Find epilines corresponding to points in right image (second image) and
    # drawing its lines on left image
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
    lines1 = lines1.reshape(-1,3)
    img5, img6 = drawlines(img1,img2,lines1,pts1,pts2)

    # Find epilines corresponding to points in left image (first image) and
    # drawing its lines on right image
    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
    lines2 = lines2.reshape(-1,3)
    img3, img4 = drawlines(img2,img1,lines2,pts2,pts1)

    #cv2.imwrite('outputs/frameL_epipolar.jpg', img5)
    #cv2.imwrite('outputs/frameR_epipolar.jpg', img6)

    """plt.subplot(121),plt.imshow(img5)
    plt.subplot(122),plt.imshow(img6)
    plt.show()"""
    
    img_center1, img_center2, center_pts1, center_pts2 = DrawAvgPoints(img1_copy, img2_copy, pts1, pts2)
    
    """plt.subplot(121),plt.imshow(img_center1)
    plt.subplot(122),plt.imshow(img_center2)
    plt.show()"""
    
    #cv2.imwrite('outputs/frameL_center.jpg', img_center1)
    #cv2.imwrite('outputs/frameR_center.jpg', img_center2)
    
    center_pts1_arr.append(list(center_pts1))
    center_pts2_arr.append(list(center_pts2))
    pts1_arr.append(list(pts1))
    pts2_arr.append(list(pts2))
    
# Save keypoints data
np.savez('data_keypoints', center_pts1_arr=center_pts1_arr, center_pts2_arr=center_pts2_arr, pts1_arr=pts1_arr, pts2_arr=pts2_arr)
print('Data of keypoints saved as "data_keypoints"!')

**SCENE 3D CREATE AND TRANSFORM**

In [7]:
# Load previously saved data of calibration
def LoadPts():
    with np.load('data_keypoints.npz') as X:
        center_pts1_arr, center_pts2_arr, pts1_arr, pts2_arr = [X[i] for i in ('center_pts1_arr', 'center_pts2_arr', 'pts1_arr', 'pts2_arr')]
    
    return center_pts1_arr, center_pts2_arr, pts1_arr, pts2_arr

def AngleDiscover(center_pts1, center_pts2, pts1, pts2):
    
    import math
    
    angle = 0
    center_pt1_dist = 0
    center_pt2_dist = 0
    
    for i in range(0, len(pts1), 1):
        center_pt1_dist += abs(center_pts1[0] - pts1[i][0])
        center_pt2_dist += abs(center_pts2[0] - pts2[i][0])
        
    print('Diferenças totais: ', center_pt1_dist, center_pt2_dist)
    
    if center_pt1_dist > center_pt2_dist:
        C = center_pt1_dist / len(pts1)
        A = center_pt2_dist / len(pts1)
    else:
        A = center_pt1_dist / len(pts1)
        C = center_pt2_dist / len(pts1)
    
    print('Distâncias cateto/hipotenusa: ', A,C)
    
    angle = math.acos(math.radians(A/C))
    
    print('Angulo de giro: ', angle)
    
    return angle

def TransformMatrix3D(df_source, frame):
    
    import math
    
    depth_angle = AngleDiscover(center_pts1_arr[frame], center_pts2_arr[frame], pts1_arr[frame], pts2_arr[frame])
    z_shift = 0
    x_center_rotation = center_pts2_arr[frame][0]
    x_shift = center_pts1_arr[frame][0] - center_pts2_arr[frame][0]
    
    #rotation_matrix = R
    #translation_matrix = t
    h = df_source.shape[0]
    
    df_rotated = df_source.copy()
        
    for i in range(0, h, 1):      
        # new Z = X.tan(angle) + depth
        df_rotated.at[i, 'z'] = (df_source.at[i, 'z'] 
                                + (math.tan(math.radians(depth_angle)) * (df_source.at[i, 'x'] - x_center_rotation)) 
                                + z_shift)
        
        df_rotated.at[i, 'x'] = df_source.at[i, 'x'] + x_shift
            
    df_source[['y','x','z']] = df_source[['y','x','z']].astype(int)
    return df_rotated

print('Funcions OK')

Funcions OK


In [11]:
### CREATE 3D SCENE WITH IMAGE AND DEPTH SAVED
import pandas as pd
import numpy as np
from PIL import Image
from pyntcloud import PyntCloud
import glob

images_left = []
images_result = []
images_left = glob.glob('outputs/frameL*.jpg')
images_disparity = glob.glob('outputs/disparity*.jpg')

center_pts1_arr, center_pts2_arr, pts1_arr, pts2_arr = LoadPts()

data_frame = pd.DataFrame()

for frame in range(0, len(images_left), 1):
    
    print('\n\033[1m' + images_left[frame] + '\033[0m')
    
    # Get the colour image. Convert the RGB values to a DataFrame
    colourImg    = Image.open(images_left[frame])
    colourPixels = colourImg.convert("RGB")

    # Add the RGB values to the DataFrame
    colourArray  = np.array(colourPixels.getdata()).reshape((colourImg.height, colourImg.width) + (3,))
    indicesArray = np.moveaxis(np.indices((colourImg.height, colourImg.width)), 0, 2)
    imageArray   = np.dstack((indicesArray, colourArray)).reshape((-1,5))
    df = pd.DataFrame(imageArray, columns=["y", "x", "red","green","blue"])

    # Open the depth-map as a greyscale image. Convert it into an array of depths. Add it to the DataFrame
    depthImg = Image.open(images_disparity[frame]).convert('L')
    depthArray = np.array(depthImg.getdata())
    df.insert(loc=2, column='z', value=depthArray)

    # Convert it to a Point Cloud and display it
    df[['y','x','z']] = df[['y','x','z']].astype(int)
    df[['red','green','blue']] = df[['red','green','blue']].astype(np.uint)

    """file_name_3d = 'outputs/result3D' + str(frame+1) + '.ply'
    cloud = PyntCloud(df)
    cloud.to_file(file_name_3d, also_save=["mesh","points"],as_text=True)"""
    
    df = FilterDataFrame(df)

    if frame > 0:
        df_transform = TransformMatrix3D(df, frame)
        data_frame = data_frame.append(df_transform, ignore_index=True)
    else:
        data_frame = data_frame.append(df, ignore_index=True)
        
cloud = PyntCloud(data_frame)
cloud.to_file("outputs/3D-SCENE.ply", also_save=["mesh","points"],as_text=True)
print('\n\nArquivo de objeto 3D salvo em "outputs/3D-SCENE.ply"!')


outputs\frameL1.jpg

outputs\frameL2.jpg
Diferenças totais:  15290 15290
Distâncias cateto/hipotenusa:  101.25827814569537 101.25827814569537
Angulo de giro:  1.5533421480573115

outputs\frameL3.jpg
Diferenças totais:  14858 14858
Distâncias cateto/hipotenusa:  99.05333333333333 99.05333333333333
Angulo de giro:  1.5533421480573115

outputs\frameL4.jpg
Diferenças totais:  15510 15510
Distâncias cateto/hipotenusa:  102.03947368421052 102.03947368421052
Angulo de giro:  1.5533421480573115

outputs\frameL5.jpg
Diferenças totais:  15826 15826
Distâncias cateto/hipotenusa:  101.44871794871794 101.44871794871794
Angulo de giro:  1.5533421480573115


Arquivo de objeto 3D salvo em "outputs/3D-SCENE.ply"!


In [10]:
def FilterDataFrame(df_filter):
    for i in range(0, len(df_filter), 1):
        if df_filter.at[i, 'z'] < 50:
            df_filter.drop([i])
    
    return df_filter

In [12]:
# VIEW THE 3D SCENE WITH PPTK
import pptk
import numpy as np
import plyfile

data = plyfile.PlyData.read('outputs/3D-SCENE.ply')['vertex']

xyz = np.c_[data['y'], data['x'], data['z']]
rgb = np.c_[data['red'], data['green'], data['blue']]

v = pptk.viewer(xyz)
v.set(point_size = 0.3)
v.attributes(rgb / 255)